In [23]:
import numpy as np
import keras
from keras import layers
import matplotlib.pyplot as plt

In [24]:
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train, x_valid = x_train_full[:-5000], x_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]


base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

In [25]:
inputs = keras.Input(shape=(32, 32, 3))

scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
resize_layer = keras.layers.Resizing(150, 150)
x = scale_layer(inputs)
x = resize_layer(x)

x = base_model(x, training=False)

x = keras.layers.GlobalAveragePooling2D()(x)

# Correcting the way the layers are defined and added
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dense(128, activation='relu')(x)
outputs = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.summary(show_trainable=True)

Model: "model_4"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_12 (InputLayer)       [(None, 32, 32, 3)]       0         Y          
                                                                            
 rescaling_5 (Rescaling)     (None, 32, 32, 3)         0         Y          
                                                                            
 resizing_5 (Resizing)       (None, 150, 150, 3)       0         Y          
                                                                            
 xception (Functional)       (None, 5, 5, 2048)        2086148   N          
                                                       0                    
                                                                            
 global_average_pooling2d_5  (None, 2048)              0         Y          
  (GlobalAveragePooling2D)                                 

In [22]:
model.compile(
    optimizer="adam",
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)
history = model.fit(x_train, y_train, epochs=10, batch_size=256, validation_data=(x_valid, y_valid))
score = model.evaluate(x_test, y_test)
print(score)

Epoch 1/10
176/176 [==============================] - 91s 499ms/step - loss: 0.5658 - accuracy: 0.8137 - val_loss: 0.4691 - val_accuracy: 0.8376
Epoch 2/10
176/176 [==============================] - 84s 479ms/step - loss: 0.3970 - accuracy: 0.8639 - val_loss: 0.4358 - val_accuracy: 0.8498
Epoch 3/10
176/176 [==============================] - 84s 477ms/step - loss: 0.3311 - accuracy: 0.8847 - val_loss: 0.4322 - val_accuracy: 0.8542
Epoch 4/10
176/176 [==============================] - 86s 488ms/step - loss: 0.2739 - accuracy: 0.9048 - val_loss: 0.4683 - val_accuracy: 0.8480
Epoch 5/10
176/176 [==============================] - 86s 488ms/step - loss: 0.2152 - accuracy: 0.9256 - val_loss: 0.4798 - val_accuracy: 0.8468
Epoch 6/10
176/176 [==============================] - 86s 488ms/step - loss: 0.1661 - accuracy: 0.9434 - val_loss: 0.4590 - val_accuracy: 0.8548
Epoch 7/10
176/176 [==============================] - 84s 478ms/step - loss: 0.1180 - accuracy: 0.9614 - val_loss: 0.5377 - val_ac